### 1) 라이브러리 설치

In [1]:
%pip install -q langchain langchain-openai langchain_community chromadb

Note: you may need to restart the kernel to use updated packages.


### 2) OpenAI 인증키 설정
https://openai.com/

In [2]:
from dotenv import load_dotenv
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

True

##### Chroma 간단한 예제
* Chroma DB에 텍스트 파일을 저장하고 검색하는 코드

In [ ]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import TextLoader  # 텍스트 파일 로더
from langchain_openai.embeddings import OpenAIEmbeddings  # OpenAI 임베딩 사용
from langchain.text_splitter import RecursiveCharacterTextSplitter  # 텍스트 분할기
from langchain_chroma import Chroma  # 벡터 DB (Chroma) 사용


# 2. 벡터 데이터베이스 저장 경로 설정
DB_PATH = "./db/chroma_db"

# 3. 텍스트 파일을 로드하고 문서를 분할하는 함수 정의
def load_and_split_text(file_path, splitter):
    """
    주어진 텍스트 파일을 로드한 후, 설정된 Splitter를 사용하여 문서를 나누는 함수.
    
    Args:
        file_path (str): 로드할 파일 경로
        splitter (RecursiveCharacterTextSplitter): 텍스트 분할기 객체

    Returns:
        list: 분할된 문서 리스트
    """
    if not os.path.exists(file_path):
        print(f" 파일을 찾을 수 없습니다: {file_path}")
        return []
    
    try:
        loader = TextLoader(file_path)  # 텍스트 파일 로드
        return loader.load_and_split(splitter)  # 분할하여 반환
    except Exception as e:
        print(f" 파일 로드 오류 ({file_path}): {e}")
        return []

# 4. 텍스트 분할기 설정 (600자 단위로 나누고, 100자 겹침 포함)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=100)

# 5. 두 개의 텍스트 파일 로드 및 분할
split_doc1 = load_and_split_text("data/ai-terminology.txt", text_splitter)
split_doc2 = load_and_split_text("data/finance-terminology.txt", text_splitter)

# 6. 문서 개수 출력
print(f"AI 문서 개수: {len(split_doc1)}")
print(f"금융 문서 개수: {len(split_doc2)}")

# 7. 모든 문서 합치기
all_documents = split_doc1 + split_doc2

# 8. Chroma 벡터 DB 생성 및 저장
try:
    persist_db = Chroma.from_documents(
        documents=all_documents,
        embedding=OpenAIEmbeddings(),  # OpenAI Embeddings 사용
        persist_directory=DB_PATH,  # 벡터 DB 저장 위치 지정
        collection_name="my_vector_db",  # 데이터베이스 컬렉션 이름
    )
    print("Chroma 데이터베이스 저장 완료!")
except Exception as e:
    print(f" Chroma 데이터베이스 저장 오류: {e}")

# 9. 저장된 데이터 확인
try:
    retrieved_docs = persist_db.get()  # Chroma DB에서 데이터 조회
    print(f" 저장된 벡터 개수: {len(retrieved_docs['ids'])}, 타입 {type(retrieved_docs['ids'])}")
except Exception as e:
    print(f" 데이터 조회 오류: {e}")

# 10. 유사도 검색 함수 정의
def search_query(query, k=2):
    """
    사용자 입력(query)에 대해 가장 유사한 문서를 검색하는 함수.

    Args:
        query (str): 검색할 문장 (예: "Transformer 개념 설명")
        k (int, optional): 검색할 문서 개수. Defaults to 2.

    Returns:
        None: 검색 결과를 출력
    """
    try:
        results = persist_db.similarity_search(query, k=k)  # 유사도 검색 수행
        print(f"\n [Query]: {query}\n")
        for i, doc in enumerate(results):
            print(f"🔹 [Result {i+1}]: {doc.page_content[:300]}...\n")  # 검색 결과 출력
    except Exception as e:
        print(f" 검색 오류: {e}")

# 11. 검색 테스트 실행
search_query("Transformer 에 대해 설명해줘", k=2)
search_query("Hedge Fund 에 대해 설명해줘?", k=2)


In [1]:
import os

from langchain_community.document_loaders import TextLoader  # 텍스트 파일 로더
from langchain_openai.embeddings import OpenAIEmbeddings  # OpenAI 임베딩 사용
from langchain.text_splitter import RecursiveCharacterTextSplitter  # 텍스트 분할기
from langchain_community.vectorstores import FAISS  # 벡터 DB (FAISS) 사용

# 2. 벡터 데이터베이스 저장 경로 설정
DB_PATH = "./db/faiss_db"

# 3. 텍스트 파일을 로드하고 문서를 분할하는 함수 정의
def load_and_split_text(file_path, splitter):
    """
    주어진 텍스트 파일을 로드한 후, 설정된 Splitter를 사용하여 문서를 나누는 함수.
    
    Args:
        file_path (str): 로드할 파일 경로
        splitter (RecursiveCharacterTextSplitter): 텍스트 분할기 객체

    Returns:
        list: 분할된 문서 리스트
    """
    if not os.path.exists(file_path):
        print(f" 파일을 찾을 수 없습니다: {file_path}")
        return []
    
    try:
        loader = TextLoader(file_path, encoding="utf-8")  # 텍스트 파일 로드
        return loader.load_and_split(splitter)  # 분할하여 반환
    except Exception as e:
        print(f" 파일 로드 오류 ({file_path}): {e}")
        return []

# 4. 텍스트 분할기 설정 (600자 단위로 나누고, 100자 겹침 포함)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=100)

# 5. 두 개의 텍스트 파일 로드 및 분할
split_doc1 = load_and_split_text("data/ai-terminology.txt", text_splitter)
split_doc2 = load_and_split_text("data/finance-terminology.txt", text_splitter)

# 6. 문서 개수 출력
print(f"AI 문서 개수: {len(split_doc1)}")
print(f"금융 문서 개수: {len(split_doc2)}")

# 7. 모든 문서 합치기
all_documents = split_doc1 + split_doc2

# 8. FAISS 벡터 DB 생성 및 저장
try:
    # FAISS 벡터 DB 생성
    persist_db = FAISS.from_documents(
        documents=all_documents,
        embedding=OpenAIEmbeddings(),  # OpenAI Embeddings 사용
    )
    
    # 로컬 디스크에 저장
    if not os.path.exists(DB_PATH):
        os.makedirs(DB_PATH)
    persist_db.save_local(DB_PATH)
    
    print("FAISS 데이터베이스 저장 완료!")
except Exception as e:
    print(f" FAISS 데이터베이스 저장 오류: {e}")

# 9. 저장된 데이터 확인
try:
    # FAISS에서 인덱스 정보 확인
    print(f" 저장된 벡터 개수: {persist_db.index.ntotal}")
    print(f" 벡터 차원: {persist_db.index.d}")
    print(f" 인덱스 타입: {type(persist_db.index)}")
except Exception as e:
    print(f" 데이터 조회 오류: {e}")

# 10. 유사도 검색 함수 정의
def search_query(query, k=2):
    """
    사용자 입력(query)에 대해 가장 유사한 문서를 검색하는 함수.

    Args:
        query (str): 검색할 문장 (예: "Transformer 개념 설명")
        k (int, optional): 검색할 문서 개수. Defaults to 2.

    Returns:
        None: 검색 결과를 출력
    """
    try:
        results = persist_db.similarity_search(query, k=k)  # 유사도 검색 수행
        print(f"\n [Query]: {query}\n")
        for i, doc in enumerate(results):
            print(f"[Result {i+1}]: {doc.page_content[:300]}...\n")  # 검색 결과 출력
    except Exception as e:
        print(f" 검색 오류: {e}")

# 11. 검색 테스트 실행
search_query("Transformer 에 대해 설명해줘", k=2)
search_query("Hedge Fund 에 대해 설명해줘?", k=2)

# 12. 저장된 FAISS DB 로드 테스트
print("\n" + "="*50)
print("저장된 FAISS DB 로드 테스트")
print("="*50)

try:
    # 저장된 FAISS DB 로드
    loaded_db = FAISS.load_local(
        DB_PATH, 
        OpenAIEmbeddings(),
        allow_dangerous_deserialization=True
    )
    print("FAISS 데이터베이스 로드 완료!")
    
    # 로드된 DB로 검색 테스트
    print(f"로드된 벡터 개수: {loaded_db.index.ntotal}")
    
    # 검색 테스트
    test_results = loaded_db.similarity_search("인공지능이란?", k=1)
    print(f"\n로드된 DB 검색 결과: {test_results[0].page_content[:200]}...")
    
except Exception as e:
    print(f"FAISS DB 로드 오류: {e}")

# 13. 유사도 점수와 함께 검색
print("\n" + "="*50)
print("유사도 점수와 함께 검색")
print("="*50)

def search_with_score(query, k=2):
    """
    유사도 점수와 함께 검색하는 함수
    """
    try:
        results = persist_db.similarity_search_with_score(query, k=k)
        print(f"\n [Query]: {query}\n")
        for i, (doc, score) in enumerate(results):
            print(f"[Result {i+1}] (Score: {score:.4f}):")
            print(f"{doc.page_content[:200]}...\n")
    except Exception as e:
        print(f" 점수 검색 오류: {e}")

# 점수와 함께 검색 테스트
search_with_score("딥러닝과 머신러닝", k=3)

# 14. FAISS 인덱스 정보 출력
print("\n" + "="*50)
print("FAISS 인덱스 상세 정보")
print("="*50)

try:
    index = persist_db.index
    print(f"인덱스 타입: {type(index).__name__}")
    print(f"총 벡터 수: {index.ntotal}")
    print(f"벡터 차원: {index.d}")
    print(f"훈련 여부: {index.is_trained}")
    print(f"메트릭 타입: {index.metric_type}")
    
    # 메모리 사용량 (추정)
    memory_usage = index.ntotal * index.d * 4 / (1024 * 1024)  # 4 bytes per float, MB 단위
    print(f"추정 메모리 사용량: {memory_usage:.2f} MB")
    
except Exception as e:
    print(f"인덱스 정보 조회 오류: {e}")

AI 문서 개수: 6
금융 문서 개수: 5
FAISS 데이터베이스 저장 완료!
 저장된 벡터 개수: 11
 벡터 차원: 1536
 인덱스 타입: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>

 [Query]: Transformer 에 대해 설명해줘

[Result 1]: Token (토큰)

정의: 텍스트 데이터를 더 작은 단위(단어, 문자, 문장 등)로 나누는 과정.
예시: "AI는 혁신적이다"를 ["AI", "는", "혁신적", "이다"]로 분할.
연관 키워드: 토큰화, NLP, 텍스트 전처리

Transformer (트랜스포머)

정의: 자연어 처리에서 사용되는 신경망 아키텍처로, 병렬 연산과 장기 의존성 처리가 강점.
예시: GPT, BERT 등의 모델이 트랜스포머 기반으로 동작함.
연관 키워드: 딥러닝, 자기 주의 메커니즘, NLP

Self-Attention (자기 주의 메커니즘)

정의...

[Result 2]: Earnings Per Share (주당순이익, EPS)

정의: 기업이 벌어들인 순이익을 총 발행 주식 수로 나눈 값.
예시: 애플의 EPS 증가로 인해 주가 상승 기대감이 커짐.
연관 키워드: 기업 실적, 투자 판단, 주식 분석

Interest Rate (이자율)

정의: 돈을 빌릴 때 지불해야 하는 비용의 비율.
예시: 연방준비제도(Fed)가 기준금리를 인상하면 대출 이자율도 상승함.
연관 키워드: 중앙은행, 채권 시장, 경제 정책

Inflation (인플레이션)

정의: 물가가 전반적으로 지속해서 상승하는 경제 현상.
예...


 [Query]: Hedge Fund 에 대해 설명해줘?

[Result 1]: Mutual Fund (뮤추얼 펀드)

정의: 여러 투자자의 자금을 모아 다양한 자산에 투자하는 펀드.
예시: 뮤추얼 펀드는 분산 투자로 위험을 줄이는 데 도움을 줌.
연관 키워드: 간접 투자, 포트폴리오, 펀드 매니저

Hedge Fund (헤지펀드)

정의: 공격적인 투자 전략을 사용하